In [1]:
import pandas as pd

In [8]:
url_2020 = 'http://www.espn.com/nba/salaries/_/seasontype/3'

In [22]:
tables_2020 = pd.read_html(url_2020)
tables_2020

[     0                       1                       2            3
 0   RK                    NAME                    TEAM       SALARY
 1    1       Stephen Curry, PG   Golden State Warriors  $40,231,758
 2    2          Chris Paul, PG   Oklahoma City Thunder  $38,506,482
 3    3   Russell Westbrook, PG         Houston Rockets  $38,506,482
 4    4           John Wall, PG      Washington Wizards  $38,199,000
 5    5        Kevin Durant, SF           Brooklyn Nets  $38,199,000
 6    6        James Harden, SG         Houston Rockets  $38,199,000
 7    7        LeBron James, SF      Los Angeles Lakers  $37,436,858
 8    8          Kyle Lowry, PG         Toronto Raptors  $34,996,296
 9    9       Blake Griffin, PF         Detroit Pistons  $34,449,964
 10  10       Tobias Harris, SF      Philadelphia 76ers  $32,742,000
 11  RK                    NAME                    TEAM       SALARY
 12  11        Jimmy Butler, SF              Miami Heat  $32,742,000
 13  12       Kawhi Leonard, SF   

In [27]:
df_2020 = tables_2020[0]
df_2020.columns = ['Rank', 'Player', 'Team', 'Salary']
df_2020 = df_2020.dropna()
df_2020

,Rank,Player,Team,Salary
0,RK,NAME,TEAM,SALARY
1,1,"Stephen Curry, PG",Golden State Warriors,"$40,231,758"
2,2,"Chris Paul, PG",Oklahoma City Thunder,"$38,506,482"
3,3,"Russell Westbrook, PG",Houston Rockets,"$38,506,482"
4,4,"John Wall, PG",Washington Wizards,"$38,199,000"
5,5,"Kevin Durant, SF",Brooklyn Nets,"$38,199,000"
6,6,"James Harden, SG",Houston Rockets,"$38,199,000"
7,7,"LeBron James, SF",Los Angeles Lakers,"$37,436,858"
8,8,"Kyle Lowry, PG",Toronto Raptors,"$34,996,296"
9,9,"Blake Griffin, PF",Detroit Pistons,"$34,449,964"


In [31]:
drop_rows = df_2020[ (df_2020['Rank'] == 'RK') & (df_2020['Player'] == 'NAME') & (df_2020['Team'] == 'TEAM') & (df_2020['Salary'] == 'SALARY')].index
df_2020.drop(drop_rows, inplace=True)
df_2020['Year'] = '2020'
df_2020

,Rank,Player,Team,Salary,Year
1,1,"Stephen Curry, PG",Golden State Warriors,"$40,231,758",2020
2,2,"Chris Paul, PG",Oklahoma City Thunder,"$38,506,482",2020
3,3,"Russell Westbrook, PG",Houston Rockets,"$38,506,482",2020
4,4,"John Wall, PG",Washington Wizards,"$38,199,000",2020
5,5,"Kevin Durant, SF",Brooklyn Nets,"$38,199,000",2020
6,6,"James Harden, SG",Houston Rockets,"$38,199,000",2020
7,7,"LeBron James, SF",Los Angeles Lakers,"$37,436,858",2020
8,8,"Kyle Lowry, PG",Toronto Raptors,"$34,996,296",2020
9,9,"Blake Griffin, PF",Detroit Pistons,"$34,449,964",2020
10,10,"Tobias Harris, SF",Philadelphia 76ers,"$32,742,000",2020


In [15]:
years = []

for i in range(2000, 2020):
    years.append(i)
years

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [54]:
# tables_all_others = []
tables_all_others_df = pd.DataFrame()

for year in years:
    url_all_others = f'http://www.espn.com/nba/salaries/_/year/{year}/seasontype/3'
    tables_all_others = pd.read_html(url_all_others)[0]
    tables_all_others['year'] = year
    tables_all_others_df = pd.concat(axis = 0, objs=[tables_all_others, tables_all_others_df])


,0,1,2,3,year
0,RK,NAME,TEAM,SALARY,2019
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2019
2,2,"Chris Paul, PG",Houston Rockets,"$35,654,150",2019
3,3,"LeBron James, SF",Los Angeles Lakers,"$35,654,150",2019
4,4,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2019


In [57]:
tables_all_others_df.head(20)


,0,1,2,3,year
0,RK,NAME,TEAM,SALARY,2019
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2019
2,2,"Chris Paul, PG",Houston Rockets,"$35,654,150",2019
3,3,"LeBron James, SF",Los Angeles Lakers,"$35,654,150",2019
4,4,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2019
5,5,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2019
6,6,"Gordon Hayward, SF",Boston Celtics,"$31,214,295",2019
7,7,"Kyle Lowry, PG",Toronto Raptors,"$31,200,000",2019
8,8,"Paul George, SG",Oklahoma City Thunder,"$30,560,700",2019
9,9,"Mike Conley, PG",Memphis Grizzlies,"$30,521,115",2019


In [56]:
tables_all_others_df.tail(20)


,0,1,2,3,year
24,22,"Bryant Reeves, C",Vancouver Grizzlies,"$8,666,000",2000
25,23,"Tom Gugliotta, PF",Phoenix Suns,"$8,558,000",2000
26,24,"Nick Van Exel, PG",Denver Nuggets,"$8,354,000",2000
27,25,"Elden Campbell, C",Charlotte Hornets,"$7,975,000",2000
28,26,"Terrell Brandon, PG",Minnesota Timberwolves,"$7,400,000",2000
29,27,"Derrick Coleman, PF",Charlotte Hornets,"$7,370,000",2000
30,28,"Glenn Robinson, SF",Milwaukee Bucks,"$7,250,000",2000
31,29,"Steve Smith, SG",Portland Trail Blazers,"$7,200,000",2000
32,30,"Kerry Kittles, SG",New Jersey Nets,"$7,166,000",2000
33,RK,NAME,TEAM,SALARY,2000


In [60]:
tables_all_others_df.columns = ['Rank', 'Player', 'Team', 'Salary', 'Year']
tables_all_others_df = tables_all_others_df.dropna()
drop_rows_others = tables_all_others_df[ (tables_all_others_df['Rank'] == 'RK') & (tables_all_others_df['Player'] == 'NAME') & (tables_all_others_df['Team'] == 'TEAM') & (tables_all_others_df['Salary'] == 'SALARY')].index
tables_all_others_df.drop(drop_rows_others, inplace=True)

In [61]:
tables_all_others_df

,Rank,Player,Team,Salary,Year
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2019
2,2,"Chris Paul, PG",Houston Rockets,"$35,654,150",2019
3,3,"LeBron James, SF",Los Angeles Lakers,"$35,654,150",2019
4,4,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2019
5,5,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2019
...,...,...,...,...,...
39,36,"Tyrone Hill, PF",Philadelphia 76ers,"$6,422,000",2000
40,37,"Clifford Robinson, PF",Phoenix Suns,"$6,125,000",2000
41,38,"Jamal Mashburn, SF",Miami Heat,"$5,600,000",2000
42,39,"Steve Nash, PG",Dallas Mavericks,"$5,500,000",2000


In [69]:
nba_salaries = pd.concat([df_2020,tables_all_others_df],ignore_index=True)
nba_salaries

,Rank,Player,Team,Salary,Year
0,1,"Stephen Curry, PG",Golden State Warriors,"$40,231,758",2020
1,2,"Chris Paul, PG",Oklahoma City Thunder,"$38,506,482",2020
2,3,"Russell Westbrook, PG",Houston Rockets,"$38,506,482",2020
3,4,"John Wall, PG",Washington Wizards,"$38,199,000",2020
4,5,"Kevin Durant, SF",Brooklyn Nets,"$38,199,000",2020
...,...,...,...,...,...
835,36,"Tyrone Hill, PF",Philadelphia 76ers,"$6,422,000",2000
836,37,"Clifford Robinson, PF",Phoenix Suns,"$6,125,000",2000
837,38,"Jamal Mashburn, SF",Miami Heat,"$5,600,000",2000
838,39,"Steve Nash, PG",Dallas Mavericks,"$5,500,000",2000


In [68]:
nba_salaries.to_csv('nba_salaries.csv', index=True)